In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey = "Bronze", key = "bronze_output")
silver_data = dbutils.jobs.taskValues.get(taskKey = "Silver", key = "silver_output")

start_date = bronze_output.get("start_date","")
silver_adls = bronze_output.get("silver_adls","")
gold_adls = bronze_output.get("gold_adls","")

print(f"Start date: {start_date}, Silver adls: {silver_adls}")

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-7945232389806477>, line 1
----> 1 bronze_output = dbutils.jobs.taskValues.get(taskKey = "bronze", key = "bronze_output")
      2 silver_data = dbutils.jobs.taskValues.get(taskKey = "silver", key = "silver_output")
      4 start_date = bronze_output.get("start_date","")

File /databricks/python_shell/lib/dbruntime/dbutils.py:208, in DBUtils.JobsHandler.TaskValuesHandler.get(self, taskKey, key, default, debugValue)
    206 elif 'NotInJobContextException' in exceptionClassName:
    207     if debugValue is None:
--> 208         raise TypeError(
    209             'Must pass debugValue when calling get outside of a job context. debugValue cannot be None.'
    210         ) from None
    211     return debugValue
    213 raise

TypeError: Must pass debugValue when calling get outside of a job context. debugValue cannot be None

In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta


In [0]:
df = spark.read.parquet(silver_data).filter(col("time") >= start_date)

In [0]:
df = df.limit(10)

In [0]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None


In [0]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())


In [0]:
get_country_code(48.8588443, 2.2943506)


Loading formatted geocoded file...
Processed coordinates: (48.8588443, 2.2943506) -> FR


'FR'

In [0]:
# adding country_code and city attributes
df_with_location = \
    df.\
    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))


In [0]:
# adding significance classification
df_with_location_sig_class = \
    df_with_location.\
    withColumn('sig_class',
        when(col("sig") < 100, "Low").\
        when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
        otherwise("High")
    )


In [0]:
df_with_location_sig_class.head()

Row(id='ak025dfawa3s', longitude=-151.6239, latitude=61.7109, elevation=83.4, title='M 1.9 - 33 km SSW of Skwentna, Alaska', place_description='33 km SSW of Skwentna, Alaska', sig=56, mag=1.9, magType='ml', time=datetime.datetime(2025, 10, 19, 23, 46, 53, 181000), updated=datetime.datetime(2025, 10, 19, 23, 50, 20, 515000), country_code='US', sig_class='Low')

In [0]:
gold_output_path = f"{gold_adls}earthquake_events_gold/"

In [0]:
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)